In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

In [ ]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

from pyspark.ml.feature import SQLTransformer
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.ml import Pipeline

In [ ]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
text_list = [
"""Les problèmes de pollution de l’eau sont importants car peuvent impacter l’accès à l’eau potable et donc la qualité de vie""",
"""Plusieurs participants constatent en revanche que ce climat sécuritaire se dégrade, même en ayant pas forcément eu de mauvaises expériences mais en écoutant leur entourage, et en se projetant sur du long terme, mais aussi pour une participante plus âgée qui fait la remarque que comparé à ce qu’elle a connu avant, cela s’est beaucoup dégradé""",
"""La localisation du logement joue beaucoup dans ce critère : « vous vous êtes centraux sur la ville, dans un endroit super sécurisé mais si tu commences à sortir un petit peu ... sur les périphéries ça devient un peu comme sur le continent » ; « En fonction du lieu de vie, en fonction d’où on habite à Ajaccio on le ressent plus ou moins »""",
"""Les critères « acquis » ne viennent pas instinctivement dans le choix lors de la construction de la pyramide : « Je me sens vraiment en sécurité donc ça m’a pas effleuré de le mettre » ; « On l’a pas mis parce que pour nous c’était une évidence qu’à Ajaccio c’était super secure »   """,
]

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

embeddings = BertSentenceEmbeddings.pretrained('labse', 'xx') \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

sentimentClassifier = ClassifierDLModel.pretrained("classifierdl_bert_sentiment", "fr") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class_")

finisher = Finisher() \
    .setInputCols(["document", "class_"])

class_exploder = SQLTransformer(statement="""
SELECT EXPLODE(arrays_zip(finished_document, class_.result, class_.metadata)) AS features
""")

result_exploder = SQLTransformer(statement="""
SELECT features['0'] AS document, features['1'] AS sentiment, features['2']['confidence'] AS confidence
""")

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        embeddings,
        sentimentClassifier,
        finisher,
        class_exploder,
        result_exploder
    ])

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import SQLTransformer
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import BertSentenceEmbeddings, ClassifierDLModel
from sparknlp import Finisher

# Initialize the Document Assembler
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

# Load pre-trained BERT sentence embeddings
embeddings = BertSentenceEmbeddings.pretrained('labse', 'xx') \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

# Load pre-trained sentiment classifier
sentimentClassifier = ClassifierDLModel.pretrained("classifierdl_bert_sentiment", "fr") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class_")

# Initialize the Finisher
finisher = Finisher() \
    .setInputCols(["document", "class_"])

# Transformer to explode arrays to handle multiple results if necessary
class_exploder = SQLTransformer(statement="""
SELECT EXPLODE(arrays_zip(finished_document, class_.result, class_.metadata)) AS features
""")

# Transformer to select and format the result fields
result_exploder = SQLTransformer(statement="""
SELECT features['0'] AS document,
       features['1'] AS sentiment,
       CAST(features['2']['confidence'] AS FLOAT) AS confidence
FROM __THIS__
""")

# Build the pipeline
nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        embeddings,
        sentimentClassifier,
        finisher,
        class_exploder,
        result_exploder
    ])

# Assuming a Spark DataFrame 'df' with a column 'text' containing the data
# Apply the pipeline to your DataFrame
model = nlpPipeline.fit(df)
result = model.transform(df)

# Show results
result.select("document", "sentiment", "confidence").show(truncate=False)


In [ ]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
(df)
result = nlpPipeline.fit(df).transform(df)


AnalysisException: Column 'finished_document' does not exist. Did you mean one of the following? []; line 2 pos 26;
'Project ['EXPLODE('arrays_zip('finished_document, 'class_.result, 'class_.metadata)) AS features#152]
+- OneRowRelation


In [ ]:
result.select(F.explode(F.arrays_zip(result.document.result,
                                     result.class_.result)).alias("cols")) \
      .select(F.expr("cols['1']").alias("class"),
              F.expr("cols['0']").alias("document")).show(truncate=False)



+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class   |document                                                                                                                                                                                                                                                                                                                                              |
+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
result.select("class_.metadata").show(truncate=False)

+-------------------------------------------------------------------+
|metadata                                                           |
+-------------------------------------------------------------------+
|[{sentence -> 0, POSITIVE -> 2.9037002E-4, NEGATIVE -> 0.99970955}]|
|[{sentence -> 0, POSITIVE -> 9.0680034E-5, NEGATIVE -> 0.9999093}] |
|[{sentence -> 0, POSITIVE -> 0.99993336, NEGATIVE -> 6.667931E-5}] |
|[{sentence -> 0, POSITIVE -> 0.9998373, NEGATIVE -> 1.626832E-4}]  |
+-------------------------------------------------------------------+

